In [2]:
import pandas as pd

df = pd.read_csv('/kaggle/input/modbus-classification/Modbus_final_final_final.csv')
columns_to_remove = ['Info', 'Protocol', 'No.', 'Time', 'Source', 'Destination', 'Length']
df = df.drop(columns=columns_to_remove)

# -1 used as padding for missing values
df.fillna(-1, inplace=True)


def hex_to_dec(x):
    x_str = str(x).strip()
    
    if all(c in '0123456789abcdefABCDEF' for c in x_str):
            return int(x_str, 16)
    else:
        return int(float(x_str))  


feature_cols = df.columns[:-1]
for col in feature_cols:
    df[col] = df[col].apply(hex_to_dec)

In [3]:
from sklearn.model_selection import train_test_split

X = df.drop('label', axis=1).values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


In [ ]:
print(X_train.shape)
print(X_test.shape)

**Attention Only**

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, Dense, Flatten, Dropout, BatchNormalization, Attention
from tensorflow.keras.models import Model
from tensorflow.keras import layers

input_layer = Input(shape=(17, 1))

attn_output = Attention()([input_layer, input_layer])
attn_output = layers.BatchNormalization()(attn_output)

conv1 = Conv1D(32, kernel_size=3, activation='relu', strides=1, padding='same')(attn_output)
conv1 = layers.BatchNormalization()(conv1)

conv2 = Conv1D(64, kernel_size=3, activation='relu', strides=1, padding='same')(conv1)
conv2 = layers.BatchNormalization()(conv2)
conv3 = Conv1D(128, kernel_size=3, activation='relu', strides=1, padding='same')(conv2)
conv3 = layers.BatchNormalization()(conv3)

flatten = Flatten()(conv3)
dense_layer1 = Dense(512, activation='relu')(flatten)
dropout_layer1 = Dropout(0.5)(dense_layer1)
dense_layer2 = Dense(256, activation='relu')(dropout_layer1)
dropout_layer2 = Dropout(0.5)(dense_layer2)

output_layer = Dense(1, activation='sigmoid')(dropout_layer2)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [6]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
 42/268 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7068 - loss: 0.8365

I0000 00:00:1716643113.664959     123 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


268/268 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.8714 - loss: 0.3802 - val_accuracy: 0.7430 - val_loss: 0.8806
Epoch 2/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9927 - loss: 0.0355 - val_accuracy: 0.9463 - val_loss: 0.1314
Epoch 3/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9950 - loss: 0.0238 - val_accuracy: 0.9963 - val_loss: 0.0250
Epoch 4/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9952 - loss: 0.0233 - val_accuracy: 0.9981 - val_loss: 0.0099
Epoch 5/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9953 - loss: 0.0190 - val_accuracy: 0.9981 - val_loss: 0.0089
Epoch 6/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9965 - loss: 0.0169 - val_accuracy: 0.9972 - val_loss: 0.0133
Epoch 7/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9966 - loss: 0.0195 - val_accuracy: 0.9981 - val_loss: 0.0093
Epoch 8/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9914 - loss: 0.0396 - val_accuracy: 0.9981 - va

In [83]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy:.2f}")

84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9956 - loss: 0.0161 
Test accuracy: 1.00


In [ ]:
attention_model = Model(inputs=model.input,
                        outputs=model.get_layer('attention').output)
attention_scores = attention_model.predict(X_test)

In [85]:
import tensorflow as tf
normalized_attention_scores_tf = tf.nn.softmax(attention_scores, axis=1).numpy()

In [ ]:
output_df_prediction_labels= model.predict(X_test)

In [102]:
import numpy as np

def hex_to_dec(hex_value):
        hex_value = hex_value.replace('0x', '')
        return int(hex_value, 16)
       

def return_packets_with_zero(path):

    output_df = pd.read_csv(path, on_bad_lines='skip')
    output_df['byte2'] = '0x0'
    output_df['byte3'] = '0x0'
    
    output_df.fillna('-1', inplace=True)
    print(output_df)
    feature_cols = output_df.columns
    for col in feature_cols:
        output_df[col] = output_df[col].apply(hex_to_dec)

   
    

    output_df_X = output_df.iloc[:, :17].values
   


    output_df_X = output_df_X.reshape(output_df_X.shape[0], output_df_X.shape[1], 1)



    output_df_prediction_labels= model.predict(output_df_X)
    output_df_prediction_labels = (output_df_prediction_labels >= 0.5).astype(int)



    output_df_X_squeezed = np.squeeze(output_df_X, axis=2)
    output_df_prediction_labels_squeezed = np.squeeze(output_df_prediction_labels)
    output_df_combined = np.column_stack((output_df_X_squeezed, output_df_prediction_labels_squeezed))


    zero_label_indices = (output_df_combined[:, -1] == 0)
    packets_with_label_zero = output_df_combined[zero_label_indices]

    packets_with_label_zero = packets_with_label_zero[:, :-1]
    packets_with_label_zero = packets_with_label_zero.reshape(packets_with_label_zero.shape[0], packets_with_label_zero.shape[1], 1)
    
    
    return packets_with_label_zero

In [103]:
def return_unique(packets):
    
    packets = np.squeeze(packets, axis=2)

    unique_packets, indices = np.unique(packets, axis=0, return_index=True)

    unique_packets = packets[np.sort(indices)]

    unique_packets = unique_packets[:, :, np.newaxis]
    
    return unique_packets

In [ ]:
packets_with_label_zero_8 = return_packets_with_zero('/kaggle/input/generated-packets-8/generated_packets_8.csv')

print("8 bytes: ", packets_with_label_zero_8.shape)

In [ ]:
unique_packets_8 = return_unique(packets_with_label_zero_8)

print("unique_packets 8 bytes: ", unique_packets_8.shape)

In [ ]:
import tensorflow as tf

attention_scores_8 = attention_model.predict(unique_packets_8)
normalized_attention_scores_tf_8 = tf.nn.softmax(attention_scores_8, axis=1).numpy()

print("8 bytes attention: ", normalized_attention_scores_tf_8.shape)

In [107]:
original_packets_8 = np.copy(unique_packets_8)

In [108]:
def flip_all_bits(byte):
    return byte ^ 0xFF

def mutate_data(unique_packets, normalized_attention_scores):
    flipped_packets_hex = []
    
    for i in range(len(unique_packets)):

        max_attention_score = np.max(normalized_attention_scores[i])

        max_indices = np.where(normalized_attention_scores[i] == max_attention_score)[0]

        for idx in max_indices:
            if idx == 5:
                continue
            byte_to_flip = unique_packets[i, idx, 0]
            flipped_byte = flip_all_bits(byte_to_flip)
            unique_packets[i, idx, 0] = flipped_byte
        
        hex_packet = ['0x' + format(x[0], '0X') for x in unique_packets[i]]
        flipped_packets_hex.append(hex_packet) 
    return flipped_packets_hex

In [109]:
mutated_8 = mutate_data(unique_packets_8, normalized_attention_scores_tf_8)

In [110]:
mutated_8_filtered = [['' if item == '0x-1' else item for item in sublist] for sublist in mutated_8]

In [111]:
df_packets_mutated_8 = pd.DataFrame(mutated_8_filtered, columns=[f'Byte_{i+1}' for i in range(17)])

df_packets_mutated_8.to_csv('df_packets_mutated_8_new_final.csv', index=False, encoding='utf-8', float_format='%.2f')

with open('mutated_packets(epoch20_20k_1).txt', 'w', encoding='utf-8') as file:
    # Write the header
    file.write('\t'.join(df_packets_mutated_8.columns) + '\n')
    
    # Write the data
    for index, row in df_packets_mutated_8.iterrows():
        formatted_row = '\t'.join(f'{value:.2f}' if isinstance(value, float) else str(value) for value in row)
        file.write(formatted_row + '\n')

In [ ]:
idx = 10
packet_flat = original_packets_8[idx].flatten()
flipped_packet_flat = unique_packets_8[idx].flatten()
attention_flat=normalized_attention_scores_tf_8[idx].flatten()
for packet_byte, attention_score, flipped in zip(packet_flat, attention_flat, flipped_packet_flat):
    print(f"{packet_byte:0X}",' : ', f"{attention_score:.8f}",' : ',f"{flipped:0X}")